### Custom Metrics SDK

Use Domino’s Custom Model Monitoring Metrics SDK to define custom metrics and use them alongside out-of-the-box drift and model quality metrics that are monitored in Domino Model Monitor. With this SDK, you can register new metrics and define the logic to compute them. You can author this logic and evaluate it from within a Domino project.

For every model that you register for monitoring, you can select a registered metric, associate the data sources from which the metric is computed, and set up the execution environment to compute this metric on a periodic basis. You are notified by email when a metric behaves abnormally based on threshold definitions.

For end-to-end working code with a description of the workflow, see the custom metrics example folder in the quick-start project


### Step 1: Instantiate the client

First, start the custom_metrics_client, and assign the custom metric to an exisitng model in Domino Model Monitoring.

In [1]:
import domino
import numpy as np
import pandas as pd
import datetime
import os
import yaml

d = domino.Domino(
    "{}/{}".format(os.environ['DOMINO_USER_NAME'], os.environ['DOMINO_PROJECT_NAME']),
    api_key=os.environ["DOMINO_USER_API_KEY"],
    host=os.environ["DOMINO_API_HOST"],
)

# Load the config file
with open("/mnt/artifacts/DMM_config.yaml") as yamlfile:
    config = yaml.safe_load(yamlfile)

# Attach alerts to the external model built in "2_External_DMM_Quickstart.ipynb" 
dmm_model_id = config['external_model_id']

# Initiate the Project
# d = domino.Domino("{}/{}".format(os.environ['DOMINO_USER_NAME'], os.environ['DOMINO_PROJECT_NAME']))
metrics_client = d.custom_metrics_client()

metrics_client

### Log the custom metrics:

**modelMonitoringId:** ID of the monitored model to send metric alerts for

**metric**: Name of the metric to send alert for

**value:** Value of the metric

**timestamp:** Timezone is in UTC in ISO 8601 format.

**tags:** Custom metadata for metric represented as key-value string pairs

### Define your custom metric

Custom Metric for Iris Use Case: Hellinger Diatance

https://en.wikipedia.org/wiki/Hellinger_distance

In [2]:
def hellinger_distance(train, inference):
    
    # distance between training data and inference data
    # train is the ditribution of an input feature in the training data
    # inference is the dsitribution of a feature being sent to the model API
    
    n = min(len(train), len(inference))
    sum = 0.0
    
    for i in range(n):
        sum += (np.sqrt(train[i]) - np.sqrt(inference[i]))**2
        
    result = (1.0 / np.sqrt(2.0)) * np.sqrt(sum)
    
    return result

#### Fetch Training set distribution for selected column

In [3]:
# Calculate the Metric
from domino.training_sets import TrainingSetClient, model

# Column we want to calculate metric for
drift_column_name = 'petal length (cm)'

# Print existing Training Sets in this Project

ts = TrainingSetClient.list_training_sets()
print(ts)

training_set = TrainingSetClient.get_training_set_version(
    training_set_name = "iris_python_multi_classification_{}".format(os.environ.get('DOMINO_PROJECT_NAME')),
    number=1
    )

training_df = training_set.load_training_pandas()
train = training_df[drift_column_name]

print(train[:5])

INFO:httpx:HTTP Request: GET http://localhost:8899/access-token "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://nucleus-frontend.domino-platform/trainingset/find?offset=0&limit=10000&asc=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8899/access-token "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://nucleus-frontend.domino-platform/trainingset/iris_python_multi_classification_monitor_workshop/1 "HTTP/1.1 200 OK"


[TrainingSet(name='iris_python_multi_classification_monitor_workshop', project_id='66ebdecd276d7e51a2b1477d', description=<training_set_api_client.types.Unset object at 0x7fd5c6b01150>, meta={})]
0    1.6
1    1.5
2    4.4
3    1.6
4    4.2
Name: petal length (cm), dtype: float64


#### Fetch Inference data

In [4]:
scoring_data = pd.read_csv('/mnt/code/data/external_model_scoring_data.csv')

inference = scoring_data[drift_column_name]

#### Calculate your metric

In [5]:
hellinger_distance = hellinger_distance(train, inference)
print('Hellinger distance between scoring and traiing data is: {}'.format(str(round(hellinger_distance, 3))))

Hellinger distance between scoring and traiing data is: 5.104


#### Log your metric with Model Monitoring

In [7]:
# Retrieve the stored metrics for the last 3 years 
import datetime
from datetime import timezone
import logging
import rfc3339

# Get time stamps for now and 1 year ago
startDate = datetime.datetime.today() - datetime.timedelta(days=365*3)
startDate = rfc3339.rfc3339(startDate)
endDate = rfc3339.rfc3339(datetime.datetime.today())

# Retrieve the metrics over the last year
try:
    res = metrics_client.read_metrics(dmm_model_id, "hellinger_distance", startDate, endDate)
    
except Exception as err:
    logging.error("Unable to fetch metrics")
    raise err
    
print(res)

ERROR:root:Unable to fetch metrics


AttributeError: DynamicSchema({'metricValues': (), 'metadata': DynamicSchema({'requestId': '768835ce-f0f3-4932-aee2-200af6115507', 'notices': ()})}) has no attribute 'metadata'

In [20]:
# timestamp = "2023-12-17T00:00:00Z"
timestamp = rfc3339.rfc3339(datetime.datetime.now()) # datetime.datetime.now().isoformat()

print(dmm_model_id)
print(hellinger_distance)
print(timestamp)
print(drift_column_name)

metrics_client.log_metric(dmm_model_id, "hellinger_distance", hellinger_distance, timestamp, { "Column" : drift_column_name})

# Sample code for logging multiple metrics
# metrics_client.log_metrics([
# { "modelMonitoringId" : dmm_model_id, "metric" : "accuracy", "value" : 7.1234,
# "timestamp" : "2022-10-08T00:00:00Z",
# "tags" : { "example_tag1" : "value1", "example_tag2" : "value2" }
# ]
# },
# { "modelMonitoringId" : dmm_model_id, "metric" : "other_metric", "value" : 8.4567,
# "timestamp" : "2022-10-09T00:00:00Z" }
# ])


66342446965e21e5b0d56d43
9
2024-05-08T21:43:20+00:00
petal length (cm)


#### Send a custom metrics alert:

**modelMonitoringId:** ID of the monitored model for which to send metric alerts.

**metric:** Name of the metric for which to send the alert.

**value:** Value of the metric.

**condition:** Target range for the metric defined by lower and upper limit bounds.
The following are potential values for the condition argument:

    metrics_client.LESS_THAN = "lessThan"

    metrics_client.LESS_THAN_EQUAL = "lessThanEqual"

    metrics_client.GREATER_THAN = "greaterThan"

    metrics_client.GREATER_THAN_EQUAL = "greaterThanEqual"

    metrics_client.BETWEEN = "between"

**lower_limit:** The lower limit for the condition.

**upper_limit:** The upper limit for the condition.

**description:** Optional message included in the alert.

In [25]:
# Set alert above threshold
dmm_model_id = config['external_model_id']
drift_column_name = 'petal length (cm)'
timestamp = rfc3339.rfc3339(datetime.datetime.now())

hellinger_distance = 9

metrics_client.log_metric(dmm_model_id, "hellinger_distance", hellinger_distance, timestamp, { "Column" : drift_column_name})


metrics_client.trigger_alert(dmm_model_id, 
                             "hellinger_distance", 
                             hellinger_distance, 
                             condition = metrics_client.BETWEEN, 
                             lower_limit=6,
                             upper_limit=8,
                             description = "Hellinger distance breached 6.0-8.0 range." 
                            )